# HADDOCK3 tutorial on modelling an antibody-antigen complex

# Introduction

This tutorial demonstrates the use of the new modular HADDOCK3 version for predicting
the structure of an antibody-antigen complex using knowledge of the hypervariable loops
on the antibody (i.e., the most basic knowledge) and epitope information identified from NMR experiments for the antigen to guide the docking.
# New Section
An antibody is a large protein that generally works by attaching itself to an antigen,
which is a unique site of the pathogen. The binding harnesses the immune system to directly
attack and destroy the pathogen. Antibodies can be highly specific while showing low immunogenicity (i.e. the ability to provoke an immune response),
which is achieved by their unique structure. **The fragment crystallizable region (Fc region)**
activates the immune response and is species-specific, i.e. the human Fc region should not
induce an immune response in humans. **The fragment antigen-binding region (Fab region**)
needs to be highly variable to be able to bind to antigens of various nature (high specificity).
In this tutorial, we will concentrate on the terminal **variable domain (Fv)** of the Fab region.

<figure style="text-align: center;">
  <img src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/antibody_described.png">
</figure>

The small part of the Fab region that binds the antigen is called **paratope**. The part of the antigen
that binds to an antibody is called **epitope**. The paratope consists of six highly flexible loops,
known as **complementarity-determining regions (CDRs)** or hypervariable loops whose sequence
and conformation are altered to bind to different antigens. CDRs are shown in red in the figure below:

<figure style="text-align: center;">
  <img src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/CDRs.png">
</figure>

In this tutorial we will be working with Interleukin-1β (IL-1β)
(PDB code [4I1B](https://www.ebi.ac.uk/pdbe/entry/pdb/4i1b)) as an antigen
and its highly specific monoclonal antibody gevokizumab
(PDB code [4G6K](https://www.ebi.ac.uk/pdbe/entry/pdb/4g6k))
(PDB code of the complex [4G6M](https://www.ebi.ac.uk/pdbe/entry/pdb/4g6m)).


---
# A brief introduction to HADDOCK3

HADDOCK3 is the next generation integrative modelling software in the
long-lasting HADDOCK project. It represents a complete rethinking and rewriting
of the HADDOCK2.X series, implementing a new way to interact with HADDOCK and
offering new features to users who can now define custom workflows.

In the previous HADDOCK2.x versions, users had access to a highly
parameterisable yet rigid simulation pipeline composed of three steps:
`rigid-body docking (it0)`, `semi-flexible refinement (it1)`, and `final refinement (itw)`.

<figure style="text-align: center;">
<img width="75%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/HADDOCK2-stages.png">
</figure>

In HADDOCK3, users have the freedom to configure docking workflows into
functional pipelines by combining the different HADDOCK3 modules, thus
adapting the workflows to their projects. HADDOCK3 has therefore developed to
truthfully work like a puzzle of many pieces (simulation modules) that users can
combine freely. To this end, the “old” HADDOCK machinery has been modularized,
and several new modules added, including third-party software additions. As a
result, the modularization achieved in HADDOCK3 allows users to duplicate steps
within one workflow (e.g., to repeat twice the `it1` stage of the HADDOCK2.x
rigid workflow).

Note that, for simplification purposes, at this time, not all functionalities of
HADDOCK2.x have been ported to HADDOCK3, which does not (yet) support NMR RDC,
PCS and diffusion anisotropy restraints, cryo-EM restraints and coarse-graining.
Any type of information that can be converted into ambiguous interaction
restraints can, however, be used in HADDOCK3, which also supports the
*ab initio* docking modes of HADDOCK.

<figure style="text-align: center;">
<img width="75%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/HADDOCK3-workflow-scheme.png">
</figure>

To keep HADDOCK3 modules organized, we catalogued them into several
categories. However, there are no constraints on piping modules of different
categories.

The main module categories are "topology", "sampling", "refinement",
"scoring", and "analysis". There is no limit to how many modules can belong to a
category. Modules are added as developed, and new categories will be created
if/when needed. You can access the HADDOCK3 documentation page for the list of
all categories and modules. Below is a summary of the available modules:

* **Topology modules**
    * `topoaa`: *generates the all-atom topologies for the CNS engine.*
* **Sampling modules**
    * `rigidbody`: *Rigid body energy minimization with CNS (`it0` in haddock2.x).*
    * `lightdock`: *Third-party glow-worm swam optimization docking software.*
* **Model refinement modules**
    * `flexref`: *Semi-flexible refinement using a simulated annealing protocol through molecular dynamics simulations in torsion angle space (`it1` in haddock2.x).*
    * `emref`: *Refinement by energy minimisation (`itw` EM only in haddock2.4).*
    * `mdref`: *Refinement by a short molecular dynamics simulation in explicit solvent (`itw` in haddock2.X).*
* **Scoring modules**
    * `emscoring`: *scoring of a complex performing a short EM (builds the topology and all missing atoms).*
    * `mdscoring`: *scoring of a complex performing a short MD in explicit solvent + EM (builds the topology and all missing atoms).*
* **Analysis modules**
    * `alascan`: *Performs a systematic (or user-define) alanine scanning mutagenesis of interface residues.*
    * `caprieval`: *Calculates CAPRI metrics (i-RMSD, l-RMSD, Fnat, DockQ) with respect to the top-scoring model or reference structure if provided.*
    * `clustfcc`: *Clusters models based on the fraction of common contacts (FCC)*
    * `clustrmsd`: *Clusters models based on pairwise RMSD matrix calculated with the `rmsdmatrix` module.*
    * `contactmap`: *Generate contact matrices of both intra- and intermolecular contacts and a chordchart of intermolecular contacts.*
    * `rmsdmatrix`: *Calculates the pairwise RMSD matrix between all the models generated in the previous step.*
    * `ilrmsdmatrix`: *Calculates the pairwise interface-ligand-RMSD (il-RMSD) matrix between all the models generated in the previous step.*
    * `seletop`: *Selects the top N models from the previous step.*
    * `seletopclusts`: *Selects the top N clusters from the previous step.*

The HADDOCK3 workflows are defined in simple configuration text files, similar to the TOML format but with extra features.
Contrary to HADDOCK2.X which follows a rigid (yet highly parameterisable)
procedure, in HADDOCK3, you can create your own simulation workflows by
combining a multitude of independent modules that perform specialized tasks.



---
# Software and data setup

In order to follow this tutorial we will start Jupyter session in Colab or other resources, install haddock3 and download the data for the tutorial.

To run this tutorial on Colab we recommand changing the runtime type to v2-8 TPU, which will give you access to up to 96 cores. The haddock3 docking run described in this tutorial should complete in about 15 minutes.

For this follow the following steps:

In [ ]:
#@title 1. Install haddock3, BioPython and py3Dmol
#@markdown Execute this cell to download the required packages.
!pip install haddock3==2025.8.0 --quiet
!pip install py3Dmol --quiet
!pip install BioPython --quiet

In [ ]:
#@title 2. Initiate custom Python functions for the 3D visualisation and alignement of the docking models { display-mode: "form" }
#@markdown Execute this cell to initiate align_full function to enable its use later in the tutorial.
import py3Dmol
import gzip
import os
from Bio.PDB import PDBParser, PDBIO, Superimposer
from Bio.PDB.Structure import Structure
from io import StringIO
import numpy as np

def load_pdb_file(file_path):
  if not os.path.exists(file_path):
    print(f"Error: File not found at {file_path}")
    return None

  if file_path.endswith('.gz'):
    with gzip.open(file_path, 'rt') as f:
      return f.read()
  else:
    with open(file_path, 'r') as f:
      return f.read()


def pdb_string_to_structure(pdb_string, structure_id):
  parser = PDBParser(QUIET=True)
  pdb_io = StringIO(pdb_string)
  structure = parser.get_structure(structure_id, pdb_io)
  return structure


def structure_to_pdb_string(structure):
  pdb_io = PDBIO()
  pdb_io.set_structure(structure)
  output = StringIO()
  pdb_io.save(output)
  return output.getvalue()

# Full alignement - user-given chains used
def align_full(pdb_path1, pdb_path2, chains=['A', 'B'], width=800, height=600,
                       model1_colors={'A': 'red', 'B': 'orange'},
                       model2_colors={'A': 'blue', 'B': 'green'},
                       atom_types=['CA'], show_labels=False, show_per_chain_rmsd=True):
    """
    Align two protein structures using all specified chains and visualize with py3Dmol.

    Parameters:
    -----------
    pdb_path1 : str
        Path to the first (reference) PDB file
    pdb_path2 : str
        Path to the second PDB file to align to the first
    chains : list, default ['A', 'B']
        List of chain IDs to include in alignment
    width : int, default 800
        Viewer width in pixels
    height : int, default 600
        Viewer height in pixels
    model1_colors : dict, default {'A': 'red', 'B': 'orange'}
        Colors for chains in model 1
    model2_colors : dict, default {'A': 'blue', 'B': 'green'}
        Colors for chains in model 2
    atom_types : list, default ['CA']
        Atom types to use for alignment (['CA'] or ['CA', 'CB', 'N', 'C'])
    show_labels : bool, default True
        Whether to show descriptive labels
    show_per_chain_rmsd : bool, default True
        Whether to calculate and display per-chain RMSD values

    Returns:
    --------
    py3Dmol.view object

    Example:
    --------
    align_full_molecule('model1.pdb.gz', 'model2.pdb.gz')
    """
    def get_atoms_from_chains(structure, chain_ids, atom_types):
        atoms = []
        chain_info = {}

        for model in structure:
            for chain_id in chain_ids:
                if chain_id in model:
                    chain = model[chain_id]
                    chain_atoms = []
                    for residue in chain:
                        for atom_type in atom_types:
                            if atom_type in residue:
                                atoms.append(residue[atom_type])
                                chain_atoms.append(residue[atom_type])
                    chain_info[chain_id] = len(chain_atoms)

        return atoms, chain_info

    # Create viewer
    view = py3Dmol.view(width=width, height=height)

    # Load PDB files
    model_1_data = load_pdb_file(pdb_path1)
    model_2_data = load_pdb_file(pdb_path2)

    if not (model_1_data and model_2_data):
        print("Failed to load one or both PDB files")
        return view, None, {}

    overall_rmsd = None
    per_chain_rmsd = {}

    try:
        # Parse structures
        struct1 = pdb_string_to_structure(model_1_data, "model1")
        struct2 = pdb_string_to_structure(model_2_data, "model2")

        # Get atoms from all specified chains
        atoms_1, chain_info_1 = get_atoms_from_chains(struct1, chains, atom_types)
        atoms_2, chain_info_2 = get_atoms_from_chains(struct2, chains, atom_types)

        print(f"Atoms for alignment - Model 1: {chain_info_1}, Total: {len(atoms_1)}")
        print(f"Atoms for alignment - Model 2: {chain_info_2}, Total: {len(atoms_2)}")
        print(f"Model 1: " + "; ".join([f"chain {chain} in {color}" for chain, color in model1_colors.items() if chain in chains]))
        print(f"Model 2: " + "; ".join([f"chain {chain} in {color}" for chain, color in model2_colors.items() if chain in chains]))

        if len(atoms_1) > 0 and len(atoms_2) > 0:
            # Align using all atoms from specified chains
            min_atoms = min(len(atoms_1), len(atoms_2))
            ref_atoms = atoms_1[:min_atoms]
            alt_atoms = atoms_2[:min_atoms]

            print(f"Using {min_atoms} atom pairs for alignment")

            # Perform superimposition
            sup = Superimposer()
            sup.set_atoms(ref_atoms, alt_atoms)
            overall_rmsd = sup.rms

            print(f"Whole molecule alignment RMSD: {overall_rmsd:.3f} Å")

            # Apply transformation to all atoms in structure 2
            sup.apply(struct2.get_atoms())

            # Calculate per-chain RMSD if requested
            if show_per_chain_rmsd:
                for chain_id in chains:
                    try:
                        chain_atoms_1, _ = get_atoms_from_chains(struct1, [chain_id], atom_types)
                        chain_atoms_2, _ = get_atoms_from_chains(struct2, [chain_id], atom_types)
                        if len(chain_atoms_1) > 0 and len(chain_atoms_2) > 0:
                            min_chain = min(len(chain_atoms_1), len(chain_atoms_2))
                            sup_chain = Superimposer()
                            sup_chain.set_atoms(chain_atoms_1[:min_chain], chain_atoms_2[:min_chain])
                            per_chain_rmsd[chain_id] = sup_chain.rms
                            print(f"Chain {chain_id} RMSD: {sup_chain.rms:.3f} Å")
                    except Exception as e:
                        print(f"Could not calculate RMSD for chain {chain_id}: {e}")

            # Convert back to PDB strings
            aligned_pdb_1 = structure_to_pdb_string(struct1)
            aligned_pdb_2 = structure_to_pdb_string(struct2)

            # Add models to viewer
            view.addModel(aligned_pdb_1, 'pdb')
            view.addModel(aligned_pdb_2, 'pdb')

        else:
            print("Could not find sufficient atoms for alignment, adding original models")
            view.addModel(model_1_data, 'pdb')
            view.addModel(model_2_data, 'pdb')

    except Exception as e:
        print(f"Alignment failed: {e}")
        view.addModel(model_1_data, 'pdb')
        view.addModel(model_2_data, 'pdb')

    # Apply styling
    view.setStyle({'model': 0}, {'cartoon': {}})
    view.setStyle({'model': 1}, {'cartoon': {}})

    for chain, color in model1_colors.items():
        if chain in chains:
            view.addStyle({'model': 0, 'chain': chain},
                         {'cartoon': {'color': color, 'opacity': 0.9}})

    for chain, color in model2_colors.items():
        if chain in chains:
            view.addStyle({'model': 1, 'chain': chain},
                         {'cartoon': {'color': color, 'opacity': 0.6}})

    # Add labels if requested
    if show_labels:
        view.addLabel("Model 1 (Reference)", {'position': {'x': -20, 'y': 20, 'z': 0},
                                             'backgroundColor': 'darkred', 'fontColor': 'white'})
        view.addLabel("Model 2 (Aligned)", {'position': {'x': 20, 'y': 20, 'z': 0},
                                           'backgroundColor': 'darkgreen', 'fontColor': 'white'})
        view.addLabel(f"Full Molecule Alignment", {'position': {'x': 0, 'y': -20, 'z': 0},
                                                  'backgroundColor': 'navy', 'fontColor': 'white'})
        if overall_rmsd:
            view.addLabel(f"Overall RMSD: {overall_rmsd:.3f} Å", {'position': {'x': 0, 'y': -30, 'z': 0},
                                                                 'backgroundColor': 'purple', 'fontColor': 'white'})

            # Add per-chain RMSD labels
            y_offset = -40
            for chain_id, rmsd_val in per_chain_rmsd.items():
                view.addLabel(f"Chain {chain_id}: {rmsd_val:.3f} Å",
                             {'position': {'x': 0, 'y': y_offset, 'z': 0},
                              'backgroundColor': 'gray', 'fontColor': 'white', 'fontSize': 10})
                y_offset -= 8

    view.zoomTo()

    return view

In [ ]:
#@title 3. Download the tutorial data
#@markdown We are providing pre-processed PDB files for docking and analysis (but the preprocessing of those files will also be explained in this tutorial). The files have been processed to facilitate their use in HADDOCK and to allow comparison with the known reference structure of the complex. Execute this cell to download and decompress the data.
!wget https://surfdrive.surf.nl/files/index.php/s/LdmwnxWFkI9l75p/download -O HADDOCK3-antibody-antigen-notebook.zip
!unzip -q HADDOCK3-antibody-antigen-notebook.zip
import os
from pathlib import Path
BASE_DIR = Path.cwd()
PROJECT_NAME = 'HADDOCK3-antibody-antigen-notebook'
PROJECT_DIR = BASE_DIR / PROJECT_NAME
os.chdir(PROJECT_DIR)


Unziping the file will create the HADDOCK3-antibody-antigen directory which should contain the following directories and files:

* pdbs: a directory containing the pre-processed PDB files
* restraints: a directory containing the interface information and the corresponding restraint files for HADDOCK3
* runs: a directory containing pre-calculated results
* scripts: a directory containing various scripts used in this tutorial
* workflows: a directory containing configuration file examples for HADDOCK3



---

# Preparing PDB files for docking

In this section we will prepare the PDB files of the antibody and antigen for docking.
Crystal structures of both the antibody and the antigen in their free forms are available from the
[PDBe database](https://www.pdbe.org).

*__Important:__ For a docking run with HADDOCK, each molecule should consist of a single chain with non-overlapping residue numbering within the same chain.

As an antibody consists of two chains (L+H), we will have to prepare it for use in HADDOCK. For this we will be making use of `pdb-tools` from the command line.

_**Note**_ that `pdb-tools` is also available as a [web service](https://wenmr.science.uu.nl/pdbtools/).


<br>

---


## Preparing the antibody structure

Using PDB-tools we will download the unbound structure of the antibody from the PDB database (the PDB ID is [4G6K](https://www.ebi.ac.uk/pdbe/entry/pdb/4g6k))
and then process it to have a unique chain ID (A) and non-overlapping residue numbering by renumbering the merged pdb (starting from 1). For this we will concatenate the following PDB-tools commands:

1. fetch the PDB entry from the PDB database (`pdb_fetch`)
2. clean the PDB file (`pdb_tidy`)
3. select the chain (`pdb_selchain`),
4. remove any hetero atoms from the structure (e.g. crystal waters, small molecules from the crystallisation buffer and such) (`pdb_delhetatm`),
5. fix residue numbering insertion in the HV loops (often occuring in antibodies - see note below) (`pdb_fixinsert`)
6. remove any possible side-chain duplication (can be present in high-resolution crystal structures in case of multiple conformations of some side chains) (`pdb_selaltloc`)
7. keep only the coordinates lines (`pdb_keepcoord`),
8. select only the variable domain (FV) of the antibody (to reduce computing time) (`pdb_selres`)
9. clean the PDB file (`pdb_tidy`)

**_Note_** that the `pdb_tidy -strict` commands cleans the PDB file, add TER statements only between different chains).
Without the -strict option TER statements would be added between each chain break (e.g. missing residues), which should be avoided.

**_Note_**: An important part for antibodies is the `pdb_fixinsert` command that fixes the residue numbering of the HV loops:
Antibodies often follow the [Chothia numbering scheme](https://pubmed.ncbi.nlm.nih.gov/9367782/?otool=inluulib)
and insertions created by this numbering scheme (e.g. 82A, 82B, 82C) cannot be processed by HADDOCK directly
(if not done those residues will not be considered resulting effectively in a break in the loop).
As such, renumbering is necessary before starting the docking.


This can be done from the command line with:

1. Fetch and process chain H
```
pdb_fetch 4G6K | pdb_tidy -strict | pdb_selchain -H | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_selres -1:120 | pdb_tidy -strict > 4G6K_H.pdb
```

2. Fetch and process chain L
```
pdb_fetch 4G6K | pdb_tidy -strict | pdb_selchain -L | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_selres -1:107 | pdb_tidy -strict > 4G6K_L.pdb
```

3. combine the heavy and light chain into one, renumbering the residues starting at 1 to avoid overlap in residue numbering between the chains and assigning a unique chainID/segID:

```
pdb_merge 4G6K_H.pdb 4G6K_L.pdb | pdb_reres -1 | pdb_chain -A | pdb_chainxseg | pdb_tidy -strict > 4G6K_clean.pdb
```

_**Note**_ The ready-to-use file can be found in the `pdbs` directory of the provided tutorial data.


In [ ]:
!pdb_fetch 4G6K | pdb_tidy -strict | pdb_selchain -H | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_selres -1:120 | pdb_tidy -strict > 4G6K_H.pdb
!pdb_fetch 4G6K | pdb_tidy -strict | pdb_selchain -L | pdb_delhetatm | pdb_fixinsert | pdb_selaltloc | pdb_keepcoord | pdb_selres -1:10 | pdb_tidy -strict > 4G6K_L.pdb
!pdb_merge 4G6K_H.pdb 4G6K_L.pdb | pdb_reres -1 | pdb_chain -A | pdb_chainxseg | pdb_tidy -strict > 4G6K_clean.pdb


---


## Preparing the antigen structure

Using PDB-tools, we will now download the unbound structure of Interleukin-1β from the PDB database (the PDB ID is [4I1B](https://www.ebi.ac.uk/pdbe/entry/pdb/4i1b)),
remove the hetero atoms and then process it to assign it chainID B.

*__Important__: Each molecule given to HADDOCK in a docking scenario must have a unique chainID/segID.*


In [ ]:
!pdb_fetch 4I1B | pdb_tidy -strict | pdb_delhetatm  | pdb_selaltloc | pdb_keepcoord | pdb_chain -B | pdb_chainxseg | pdb_tidy -strict > 4I1B_clean.pdb



---


# Defining restraints for docking

Before setting up the docking, we first need to generate distance restraint files in a format suitable for HADDOCK.
HADDOCK uses [CNS][link-cns]{:target="_blank"} as computational engine.
A description of the format for the various restraint types supported by HADDOCK can be found in our [Nature Protocol 2024][nat-pro]{:target="_blank"} paper, Box 1.

Distance restraints are defined as follows:

<pre>
assign (selection1) (selection2) distance, lower-bound correction, upper-bound correction
</pre>

The lower limit for the distance is calculated as: distance minus lower-bound correction
and the upper limit as: distance plus upper-bound correction.

The syntax for the selections can combine information about:

* chainID - `segid` keyword
* residue number - `resid` keyword
* atom name - `name` keyword.

Other keywords can be used in various combinations of OR and AND statements. Please refer for that to the [online CNS manual][link-cns]{:target="_blank"}.

E.g.: a distance restraint between the CB carbons of residues 10 and 200 in chains A and B with an
allowed distance range between 10Å and 20Å would be defined as follows:

<pre>
assign (segid A and resid 10 and name CB) (segid B and resid 200 and name CB) 20.0 10.0 0.0
</pre>

__*Can you think of a different way of defining the distance and lower and upper corrections while maintaining the same allowed range?*__




### Identifying the paratope of the antibody

Nowadays several computational tools can identify the paratope (the residues of the hypervariable loops involved in binding) from the provided antibody sequence.
In this tutorial, we are providing you with the corresponding list of residue obtained using [ProABC-2](https://github.com/haddocking/proabc-2).
ProABC-2 uses a convolutional neural network to identify not only residues which are located in the paratope region but also the nature of interactions they are most likely involved in (hydrophobic or hydrophilic).
The work is described in [Ambrosetti, *et al* Bioinformatics, 2020](https://academic.oup.com/bioinformatics/article/36/20/5107/5873593).

The corresponding paratope residues (those with either an overall probability >= 0.4 or a probability for hydrophobic or hydrophilic > 0.3) are:

<pre>
31,32,33,34,35,52,54,55,56,100,101,102,103,104,105,106,151,152,169,170,173,211,212,213,214,216
</pre>

The numbering corresponds to the numbering of the `4G6K_clean.pdb` PDB file.

Let us visualize those onto the 3D structure. For this execute the next cell.


In [ ]:
#@title Visualize the antibody paratope
import py3Dmol
import gzip
import os

# Create a 3Dmol viewer object
view = py3Dmol.view(width=800, height=600)

# Specify the path to your PDB file (can be .pdb or .pdb.gz)
pdb_file_path = PROJECT_DIR / "pdbs" / "4G6K_clean.pdb"

# Check if the file exists
if not os.path.exists(pdb_file_path):
    print(f"Error: File not found at {pdb_file_path}")
else:
    # Check if the file is gzipped
    if str(pdb_file_path).endswith('.gz'):
        with gzip.open(pdb_file_path, 'rt') as f:
            view.addModel(f.read(), 'pdb')
    else:
        with open(pdb_file_path, 'r') as f:
            view.addModel(f.read(), 'pdb')

    view.setStyle({'cartoon': {'color': 'white'}})
    view.addStyle({'stick': {'color': 'white'}})
    view.addStyle({'resi': [31,32,33,34,35,52,54,55,56,100,101,102,103,104,105,106,151,152,169,170,173,211,212,213,214,216]},{'cartoon':{'color':'red'}})
    view.addStyle({'resi': [31,32,33,34,35,52,54,55,56,100,101,102,103,104,105,106,151,152,169,170,173,211,212,213,214,216]},{'stick':{'color':'red'}})
    view.addSurface(py3Dmol.SAS, {'opacity':0.4, 'color':'white'})
    view.zoomTo()
    view.show()


__*Do the identified paratope residues form a well-defined patch on the surface?*__


<details style="background-color:#DAE4E7">
  <summary style="bold">
    <b><i>See surface view of the paratope</i></b>
  </summary>
  <figure style="text-align: center;">
    <img width="60%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/antibody-paratope.png">
  </figure>
  <br>
</details>





---


## Identifying the epitope of the antigen

The article describing the crystal structure of the antibody-antigen complex we are modeling also reports experimental NMR chemical shift titration experiments
to map the binding site of the antibody (gevokizumab) on Interleukin-1β.
The residues affected by binding are listed in Table 5 of [Blech et al. JMB 2013](https://dx.doi.org/10.1016/j.jmb.2012.09.021):

<figure style="text-align: center;">
  <img width="40%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/Table5-Blech.png">
</figure>

The list of binding site (epitope) residues identified by NMR is:

<pre>
72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117
</pre>

We will now visualize the epitope on Interleukin-1β.


In [ ]:
#@title Visualize the NMR-defined epitope
import py3Dmol
import gzip
import os

# Create a 3Dmol viewer object
view = py3Dmol.view(width=800, height=600)

# Specify the path to your PDB file (can be .pdb or .pdb.gz)
pdb_file_path = './pdbs/4I1B_clean.pdb'

# Check if the file exists
if not os.path.exists(pdb_file_path):
    print(f"Error: File not found at {pdb_file_path}")
else:
    # Check if the file is gzipped
    if pdb_file_path.endswith('.gz'):
        with gzip.open(pdb_file_path, 'rt') as f:
            view.addModel(f.read(), 'pdb')
    else:
        with open(pdb_file_path, 'r') as f:
            view.addModel(f.read(), 'pdb')

    view.setStyle({'cartoon': {'color': 'white'}})
    view.addStyle({'stick': {'color': 'white'}})
    view.addStyle({'resi': [72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117]},{'cartoon':{'color':'red'}})
    view.addStyle({'resi': [72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117]},{'stick':{'color':'red'}})
    view.addSurface(py3Dmol.SAS, {'opacity':0.4, 'color':'white'})
    view.zoomTo()
    view.show()

Inspect the surface.

The answer to that question should be yes, but we can see some residues not colored that might also be involved in the binding - there are some white spots around/in the red surface.


__*Do the identified residues form a well-defined patch on the surface?*__

<details style="background-color:#DAE4E7">
  <summary style="bold">
    <b><i>See surface view of the epitope identified by NMR</i></b>
  </summary>
  <figure style="text-align: center;">
    <img width="60%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/antigen-epitope.png">
  </figure>
  <br>
</details>




---

## Defining surface neighbours as passive residues

In HADDOCK, we are dealing with potentially incomplete binding sites by defining surface neighbors as `passive` residues.
These passive residues are added in the definition of the interface but do not incur any energetic penalty if they are not part of the binding site in the final models.
In contrast, residues defined as active (typically the identified or predicted binding site residues) will incur an energetic penalty.
When using the HADDOCK2.x webserver, `passive` residues will be automatically defined.
Here, since we are using a local version, we need to define those manually.

This can easily be done using a haddock3 command line tool in the following way:

```
haddock3-restraints passive_from_active 4I1B_clean.pdb 72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117
```

The command prints a list of solvent accessible passive residues, which you should save to a file for further use.


In [ ]:
!haddock3-restraints passive_from_active 4I1B_clean.pdb 72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117

In [ ]:
#@title Visualise the NMR-defined epitope with surface neighbours
import py3Dmol
import gzip
import os

# Create a 3Dmol viewer object
view = py3Dmol.view(width=800, height=600)

# Specify the path to your PDB file (can be .pdb or .pdb.gz)
pdb_file_path = PROJECT_DIR / "pdbs" / "4I1B_clean.pdb"

# Check if the file exists
if not os.path.exists(pdb_file_path):
    print(f"Error: File not found at {pdb_file_path}")
else:
    # Check if the file is gzipped
    if str(pdb_file_path).endswith('.gz'):
        with gzip.open(pdb_file_path, 'rt') as f:
            view.addModel(f.read(), 'pdb')
    else:
        with open(pdb_file_path, 'r') as f:
            view.addModel(f.read(), 'pdb')

    view.setStyle({'cartoon': {'color': 'white'}})
    view.addStyle({'stick': {'color': 'white'}})
    view.addStyle({'resi': [72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117]},{'cartoon':{'color':'red'}})
    view.addStyle({'resi': [72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117]},{'stick':{'color':'red'}})
    view.addStyle({'resi': [3,24,46,47,48,50,66,76,77,79,80,82,86,87,88,91,93,95,118,119,120]},{'cartoon':{'color':'green'}})
    view.addStyle({'resi': [3,24,46,47,48,50,66,76,77,79,80,82,86,87,88,91,93,95,118,119,120]},{'stick':{'color':'green'}})
    view.addSurface(py3Dmol.SAS, {'opacity':0.4, 'color':'white'})
    view.zoomTo()
    view.show()

<details style="background-color:#DAE4E7">
  <summary style="bold">
    <b><i>See the epitope and passive residues</i></b>
  </summary>
  <figure style="text-align: center;">
    <img width="60%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/antigen-active-passive.png">
  </figure>
  <br>
</details>
<br>

The NMR-identified residues and their surface neighbors generated with the above command can be used to define ambiguous interactions restraints,
either using the NMR identified residues as active in HADDOCK, or combining those with the surface neighbors.

The difference between `active` and `passive` residues in HADDOCK is as follows:

*__Active residues__*: These residues are "forced" to be at the interface. If they are not part of the interface in the final models, an energetic penalty will be applied. The interface in this context is defined by the union of active and passive residues on the partner molecules.

*__Passive residues__*: These residues are expected to be at the interface. However, if they are not, no energetic penalty is applied.


In general, it is better to be too generous rather than too strict in the definition of passive residues.
An important aspect is to filter both the active (the residues identified from your mapping experiment) and passive residues by their solvent accessibility.
This is done automatically when using the `haddock3-restraints passive_from_active` command: residues with less that 15% relative solvent accessibility (same cutoff as the default in the HADDOCK server) are discared.
This is, however, not a hard limit, and you might consider including even more buried residues if some important chemical group seems solvent accessible from a visual inspection.


---

## Defining ambiguous restraints

Once you have identified your active and passive residues for both molecules, you can proceed with the generation of the ambiguous interaction restraints (AIR) file for HADDOCK.
For this you can either make use of our online [Generate Restraints](https://rascar.science.uu.nl/haddock-restraints) web service, entering the list of active and passive residues for each molecule, the chainIDs of each molecule and saving the resulting restraint list to a text file, or use another `haddock3-restraints` sub-command.

To use our `haddock3-restraints active_passive_to_ambig` script, you need to
create for each molecule a file containing two lines:

* The first line corresponds to the list of active residues (numbers separated by spaces)
* The second line corresponds to the list of passive residues (numbers separated by spaces).

*__Important__*: The file must consist of two lines, but a line can be empty (e.g., if you do not want to define active residues for one molecule).
However, there must be at least one set of active residue defined for one of the molecules.


* For the antibody we will use the predicted paratope as active and no passive residues defined.
The corresponding file can be found in the `restraints` directory as `antibody-paratope.act-pass`:

```
1 32 33 34 35 52 54 55 56 100 101 102 103 104 105 106 151 152 169 170 173 211 212 213 214 216

```

* For the antigen we will use the NMR-identified epitope as active and the surface neighbors as passive.
The corresponding file can be found in the `restraints` directory as `antigen-NMR-epitope.act-pass`:

```
72 73 74 75 81 83 84 89 90 92 94 96 97 98 115 116 117
3 24 46 47 48 50 66 76 77 79 80 82 86 87 88 91 93 95 118 119 120
```

Using those two files, we can generate the CNS-formatted Ambiguous Interaction Restraints (AIRs) file with the following command:

```
haddock3-restraints active_passive_to_ambig restraints/antibody-paratope.act-pass restraints/antigen-NMR-epitope.act-pass --segid-one A --segid-two B > ambig-paratope-NMR-epitope.tbl
```

This generates a file called `ambig-paratope-NMR-epitope.tbl` that contains the AIRs.


In [ ]:
!haddock3-restraints active_passive_to_ambig restraints/antibody-paratope.act-pass restraints/antigen-NMR-epitope.act-pass --segid-one A --segid-two B > ambig-paratope-NMR-epitope.tbl


__*Inspect the generated file and note how the ambiguous distances are defined.*__

<details style="background-color:#DAE4E7">
  <summary style="bold">
    <b><i>View an extract of the AIR file</i></b>
  </summary>
<pre>
assign (resi 31 and segid A)
(
       (resi 72 and segid B)
        or
       (resi 73 and segid B)
        or
       (resi 74 and segid B)
        or
       (resi 75 and segid B)
        or
       (resi 81 and segid B)
        or
       (resi 83 and segid B)
        or
       (resi 84 and segid B)
        or
       (resi 89 and segid B)
        or
       (resi 90 and segid B)
        or
       (resi 92 and segid B)
        or
       (resi 94 and segid B)
        or
       (resi 96 and segid B)
        or
       (resi 97 and segid B)
        or
       (resi 98 and segid B)
        or
       (resi 115 and segid B)
        or
       (resi 116 and segid B)
        or
       (resi 117 and segid B)
        or
       (resi 3 and segid B)
        or
       (resi 24 and segid B)
        or
       (resi 46 and segid B)
        or
       (resi 47 and segid B)
        or
       (resi 48 and segid B)
        or
       (resi 50 and segid B)
        or
       (resi 66 and segid B)
        or
       (resi 76 and segid B)
        or
       (resi 77 and segid B)
        or
       (resi 79 and segid B)
        or
       (resi 80 and segid B)
        or
       (resi 82 and segid B)
        or
       (resi 86 and segid B)
        or
       (resi 87 and segid B)
        or
       (resi 88 and segid B)
        or
       (resi 91 and segid B)
        or
       (resi 93 and segid B)
        or
       (resi 95 and segid B)
        or
       (resi 118 and segid B)
        or
       (resi 119 and segid B)
        or
       (resi 120 and segid B)
) 2.0 2.0 0.0
...
</pre>
  <br>
</details>
<br>

__*Refering to the way the distance restraints are defined (see above), what is the distance range for the ambiguous distance restraints?*__

<details style="background-color:#DAE4E7">
  <summary style="bold">
    <b><i>See answer</i></b>
  </summary>
The default distance range for those is between 0 and 2Å, which
might seem short but makes senses because of the 1/r^6 summation in the AIR
energy function that makes the effective distance to be significantly shorter than the shortest distance entering the sum.
<br>
<br>
The effective distance is calculated as the SUM over all pairwise atom-atom
distance combinations between an active residue and all the active+passive on
the other molecule: SUM[1/r^6]^(-1/6).
</details>
<br>




---


## Restraints validation

If you modify manually this generated restraint files or create your own, it is possible to quickly check if the format is valid using the following `haddock3-restraints` sub-command:

```
haddock3-restraints validate_tbl ambig-paratope-NMR-epitope.tbl --silent
```

No output means that your TBL file is valid.

*__Note__* that this only validates the syntax of the restraint file, but does not check if the selections defined in the restraints are actually existing in your input PDB files.



In [ ]:
!haddock3-restraints validate_tbl ambig-paratope-NMR-epitope.tbl --silent



---

## Additional restraints for multi-chain proteins

As an antibody consists of two separate chains, it is important to define a few distance restraints
to keep them together during the high temperature flexible refinement stage of HADDOCK otherwise they might slightly drift appart.
This can easily be done using the `haddock3-restraints restrain_bodies` sub-command.


In [ ]:
!haddock3-restraints restrain_bodies pdbs/4G6K_clean.pdb > antibody-unambig.tbl


The result file contains two CA-CA distance restraints with the exact distance measured between two randomly picked CA atoms pairs:

```  
assign (segid A and resi 110 and name CA) (segid A and resi 132 and name CA) 26.326 0.0 0.0
assign (segid A and resi 97 and name CA) (segid A and resi 204 and name CA) 19.352 0.0 0.0
```

This file is also provided in the `restraints` directory.





---


# Setting up and running the docking with HADDOCK3

Now that we have all required files at hand (PDB and restraints files), it is time to setup our docking protocol.
In this tutorial, considering we have rather good information about the paratope and epitope, we will execute a fast HADDOCK3 docking workflow,
reducing the non-negligible computational cost of HADDOCK by decreasing the sampling, without impacting too much the accuracy of the resulting models.






---


## HADDOCK3 workflow definition

The first step is to create a HADDOCK3 configuration file that will define the docking workflow.
We will follow a classic HADDOCK workflow consisting of rigid body docking, semi-flexible refinement and final energy minimisation followed by clustering.

We will also integrate two analysis modules in our workflow:

- `caprieval` will be used at various stages to compare models to either the best scoring model (if no reference is given) or a reference structure, which in our case we have at hand (`pdbs/4G6M_matched.pdb`). This will directly allow us to assess the performance of the protocol. In the absence of a reference, `caprieval` is still usefull to assess the convergence of a run and analyse the results.
- `contactmap` added as last module will generate contact matrices of both intra- and intermolecular contacts and a chordchart of intermolecular contacts for each cluster.


Our workflow consists of the following modules:

1. **`topoaa`**: *Generates the topologies for the CNS engine and builds missing atoms*
2. **`rigidbody`**: *Performs rigid body energy minimisation (`it0` in haddock2.x)*
3. **`caprieval`**: *Calculates CAPRI metrics (i-RMSD, l-RMSD, Fnat, DockQ) with respect to the top scoring model or reference structure if provided*
4. **`seletop`** : *Selects the top X models from the previous module*
5. **`flexref`**: *Preforms semi-flexible refinement of the interface (`it1` in haddock2.4)*
6. **`caprieval`**
7. **`emref`**: *Final refinement by energy minimisation (`itw` EM only in haddock2.4)*
8. **`caprieval`**
9. **`clustfcc`**: *Clustering of models based on the fraction of common contacts (FCC)*
10. **`seletopclusts`**: *Selects the top models of all clusters*
11. **`caprieval`**
12. **`contactmap`**: *Contacts matrix and a chordchart of intermolecular contacts*


The corresponding toml configuration file (provided in `workflows/docking-antibody-antigen.cfg`) looks like:

```
# ====================================================================
# Antibody-antigen docking example with restraints from the antibody
# paratope to the NMR-identified epitope on the antigen
# ====================================================================

# Directory in which the scoring will be done
run_dir = "run1"

# Compute mode
mode = "local"
ncores = 50

# Self contained rundir (to avoid problems with long filename paths)
self_contained = true

# Post-processing to generate statistics and plots
postprocess = true
clean = true

molecules =  [
    "pdbs/4G6K_clean.pdb",
    "pdbs/4I1B_clean.pdb"
    ]

# ====================================================================
# Parameters for each stage are defined below, prefer full paths
# ====================================================================
[topoaa]

[rigidbody]
# CDR to NMR epitope ambig restraints
ambig_fname = "restraints/ambig-paratope-NMR-epitope.tbl"
# Restraints to keep the antibody chains together
unambig_fname = "restraints/antibody-unambig.tbl"
# Reduced sampling (100 instead of the default of 1000)
sampling = 100

[caprieval]
reference_fname = "pdbs/4G6M_matched.pdb"

[seletop]
# Selection of the top 20 best scoring complexes (instead of the default of 200 for speed purposes for the tutorial)
select = 20

[flexref]
tolerance = 5
# CDR to NMR epitope ambig restraints
ambig_fname = "restraints/ambig-paratope-NMR-epitope.tbl"
# Restraints to keep the antibody chains together
unambig_fname = "restraints/antibody-unambig.tbl"

[caprieval]
reference_fname = "pdbs/4G6M_matched.pdb"

[emref]
tolerance = 5
# CDR to NMR epitope ambig restraints
ambig_fname = "restraints/ambig-paratope-NMR-epitope.tbl"
# Restraints to keep the antibody chains together
unambig_fname = "restraints/antibody-unambig.tbl"

[caprieval]
reference_fname = "pdbs/4G6M_matched.pdb"

[clustfcc]
plot_matrix = true

[seletopclusts]
# Selection of the top 4 best scoring complexes from each cluster
top_models = 4

[caprieval]
reference_fname = "pdbs/4G6M_matched.pdb"

[contactmap]

# ====================================================================

```


In this case, since we have information for both interfaces we use a low-sampling configuration file, which takes only a small amount of computational resources to run.
The initial `sampling` parameter at the rigid-body energy minimization (`rigidbody`) module is set to 100 models, of which only best the 40 are passed to the flexible refinement (`flexref`) module with the `seletop` module.
The subsequence flexible refinement (`flexref` module) and energy minimisation (*emref*) modules will use all models passed by the *seletop* module.
FCC clustering (`clustfcc`) is then applied to group together models sharing a consistent fraction of the interface contacts.
The top 4 models of each cluster are saved to disk (`seletopclusts`).

Multiple `caprieval` modules are executed at different stages of the workflow to check how the quality (and rankings) of the models change throughout the protocol.
In this case we are providing the known crystal structure of the complex as reference.


**_Note_**: For making best use of the available CPU resources it is recommended to adapt the sampling parameter to be a multiple of the number of available cores when running in local mode. For this reason, for the ASEAN HPC school the sampling is set to be a multiple of 48.

**_Note_**: In case no reference is available (the usual scenario), the best ranked model is used as reference for each stage.
Including `caprieval` at the various stages even when no reference is provided is useful to get the rankings and scores and visualise the results (see Analysis section below).

**_Note_**: The default sampling would be 1000 models for `rigidbody` of which 200 are passed to the flexible refinement in `seletop`.
As an indication of the computational requirements, the default sampling worflow for this tutorial completes in about 37min using 12 cores on a MaxOSX M2 processor.
In comparison, the reduced sampling run (100/40) takes about 7-8min.



**_Note_**: To get a list of all possible parameters that can be defined in a specific module (and their default values) you can use the following command:

```
haddock3-cfg -m <module-name>
```
Add the `-d` option to get a more detailed description of parameters and use the `-h` option to see a list of arguments and options.

Alternatively, you can consult the [developer's guide](https://www.bonvinlab.org/haddock3/), where each parameter of each module is listed along with their default values, short and long descriptions, etc. Navigate to the [Modules](https://www.bonvinlab.org/haddock3/modules/index.html#) and select a module which parameters you want to display.

__*In the above workflow we see in three modules a *tolerance* parameter defined. Using the `haddock3-cfg` command try to figure out what this parameter does.*__



**_Note_** that, in contrast to HADDOCK2.X, we have much more flexibility in defining our workflow.
As an example, we could use this flexibility by introducing a clustering step after the initial rigid-body docking stage, selecting a given number of models per cluster and refining all of those.
For an example of this strategy see the section about ensemble docking.




---

## Running HADDOCK3

To execute the above workflow you have two choices:

1. __Open a terminal session__ and simply type (check if the location is correct):

```
cd HADDOCK3-antibody-antigen-notebook
haddock3 workflows/docking-antibody-antigen.cfg >&run.out &
```

This will run haddock3 in background. In the mean time we can contine with the analysis section using the precalculated results.

You can check the status of the run by looking at the `run.out` file.

2. __Or execute the next cell__, and wait for its completion to proceed with the analysis (time for a cup of coffee :-) as the run should take about 15 minutes to complete)

In [ ]:
!haddock3 workflows/docking-antibody-antigen.cfg



In case the execution was interupted you can restart haddock3 from the last completed stage. Check the content of the run1 directory and find out which stage was the last running and not completed. Then restart haddock3 with as addional option: ``--restart 4`` (for example to restart at stage 4). This can be done from the terminal within the `HADDOCK3-antibody-antigen-notebook` directory.

```
haddock3 workflows/docking-antibody-antigen.cfg --restart 4
```



---


# Analysis of the docking results

In case something went wrong with the docking (or simply if you do not want to wait for the results) you can find the following precalculated runs in the `runs` directory:
- `run1`: docking run created using the unbound antibody.
- `run-energetics`: alanine scanning run (see BONUS1 section)
- `run-scoring`: combined scoring of HADDOCK, AF2 and AF3 models  (see BONUS2 section).


Once your run has completed - inspect the content of the resulting directory.
You will find the various steps (modules) of the defined workflow numbered sequentially starting at 0, e.g.:

```
> ls run1/
     00_topoaa/
     01_rigidbody/
     02_caprieval/
     03_seletop/
     04_flexref/
     05_caprieval/
     06_emref/
     07_caprieval/
     08_clustfcc/
     09_seletopclusts/
     10_caprieval/
     11_contactmap/
     analysis/
     data/
     log
     toppar/
     traceback/
```

In addition, there is a log file (text file) and four additional directories:

- the `analysis` directory contains various plots to visualize the results for each caprieval step and a general report (`report.html`) that provides all statistics with various plots. You can open this file in your preferred web browser
- the `data` directory contains the input data (PDB and restraint files) for the various modules, as well as an input workflow  (in `configurations` directory)
- the `toppar` directory contains the force field topology and parameter files (only present when running in self-contained mode)
- the `traceback` directory contains `traceback.tsv`, which links all models to see which model originates from which throughout all steps of the workflow.

You can find information about the duration of the run at the bottom of the log file.

Each sampling/refinement/selection module will contain PDB files.
For example, the `09_seletopclusts` directory contains the selected models from each cluster. The clusters in that directory are numbered based
on their rank, i.e. `cluster_1` refers to the top-ranked cluster. Information about the origin of these files can be found in that directory in the `seletopclusts.txt` file.

The simplest way to extract ranking information and the corresponding HADDOCK scores is to look at the `XX_caprieval` directories (which is why it is a good idea to have it as the final module, and possibly as intermediate steps). This directory will always contain a `capri_ss.tsv` single model statistics file, which contains the model names, rankings and statistics (score, iRMSD, Fnat, lRMSD, ilRMSD and dockq score). E.g. for `10_caprieval`:

```
model   md5     caprieval_rank  score   irmsd   fnat    lrmsd   ilrmsd  dockq   rmsd    cluster_id      cluster_ranking model-cluster_ranking   air     angles  bonds   bsa     cdih    coup    dani    desolv  dihe    elec    improper        rdcs    rg      sym     total   vdw     vean    xpcs
../../09_seletopclusts/cluster_1_model_1.pdb    -       1       -129.035        1.351   0.862   3.731   2.772   0.751   1.557   2       1       1       111.427 278.615 52.004  1799.440        0.000   0.000   0.000   0.725   2042.510        -437.132        72.945  0.000   0.000   0.000   -379.180     -53.476 0.000   0.000
../../09_seletopclusts/cluster_1_model_2.pdb    -       2       -114.462        1.502   0.810   3.387   3.100   0.724   1.498   2       1       2       144.192 281.411 52.711  1880.050        0.000   0.000   0.000   3.131   2041.260        -458.989        75.587  0.000   0.000   0.000   -355.011     -40.214 0.000   0.000
../../09_seletopclusts/cluster_1_model_3.pdb    -       3       -114.217        1.296   0.776   4.068   2.974   0.721   1.350   2       1       3       132.033 283.299 51.917  1872.550        0.000   0.000   0.000   8.364   2050.640        -510.529        73.423  0.000   0.000   0.000   -412.174     -33.678 0.000   0.000
...
```

If clustering was performed prior to calling the `caprieval` module, the `capri_ss.tsv` file will also contain information about to which cluster the model belongs to and its ranking within the cluster.

The relevant statistics are:

* **score**: *the HADDOCK score (arbitrary units)*
* **irmsd**: *the interface RMSD, calculated over the interfaces the molecules*
* **fnat**: *the fraction of native contacts*
* **lrmsd**: *the ligand RMSD, calculated on the ligand after fitting on the receptor (1st component)*
* **ilrmsd**: *the interface-ligand RMSD, calculated over the interface of the ligand after fitting on the interface of the receptor (more relevant for small ligands for example)*
* **dockq**: *the DockQ score, which is a combination of irmsd, lrmsd and fnat and provides a continuous scale between 1 (exactly equal to reference) and 0*

Various other terms are also reported including:

* **bsa**: *the buried surface area (in squared angstroms)*
* **elec**: *the intermolecular electrostatic energy*
* **vdw**: *the intermolecular van der Waals energy*
* **desolv**: *the desolvation energy*


The iRMSD, lRMSD and Fnat metrics are the ones used in the blind protein-protein prediction experiment [CAPRI](https://www.ebi.ac.uk/msd-srv/capri/) (Critical PRediction of Interactions).

In CAPRI the quality of a model is defined as (for protein-protein complexes):

* **acceptable model**: i-RMSD < 4Å or l-RMSD < 10Å and Fnat > 0.1 (0.23 < DOCKQ < 0.49)
* **medium quality model**: i-RMSD < 2Å or l-RMSD < 5Å and Fnat > 0.3 (0.49 < DOCKQ < 0.8)
* **high quality model**: i-RMSD < 1Å or l-RMSD < 1Å and Fnat > 0.5 (DOCKQ > 0.8)

__*Based on these CAPRI criteria, what is the quality of the best model listed above (_cluster_1_model_1.pdb_)?*__


In case where the `caprieval` module is called after a clustering step, an additional `capri_clt.tsv` file will be present in the directory.
This file contains the cluster ranking and score statistics, averaged over the minimum number of models defined for clustering
(4 by default), with their corresponding standard deviations. E.g.:

```
cluster_rank    cluster_id      n       under_eval      score   score_std       irmsd   irmsd_std       fnat    fnat_std        lrmsd   lrmsd_std       dockq   dockq_std       ilrmsd  ilrmsd_std      rmsd    rmsd_std        air     air_std bsa     bsa_std desolv  desolv_std      elec    elec_std     total   total_std       vdw     vdw_std caprieval_rank
1       2       3       yes     -119.238 6.928   1.383   0.087   0.816   0.035   3.729   0.278   0.732   0.013   2.949   0.135   1.468   0.087   129.217 13.524  1850.680        36.361  4.073   3.189   -468.883        30.770  -382.122        23.429  -42.456 8.236   1
2       1       4       -        -97.753 7.662   5.037   0.124   0.134   0.007   12.077  0.969   0.183   0.014   10.874  0.328   4.494   0.261   166.356 17.690  1585.855        62.659  5.596   2.998   -272.086        38.750  -171.297        33.845  -65.568 8.237   2
3       3       2       yes      -84.236 8.511   10.756  0.411   0.069   0.017   20.253  0.596   0.079   0.003   18.837  0.562   11.044  0.347   136.880 11.285  1397.195        38.715  5.962   0.830   -327.803        47.780  -229.249        56.062  -38.325 3.004   3
```


In this file you find the cluster rank (which corresponds to the naming of the clusters in the previous `seletop` directory), the cluster ID (which is related to the size of the cluster, 1 being always the largest cluster), the number of models (n) in the cluster and the corresponding statistics (averages + standard deviations). The corresponding cluster PDB files will be found in the preceeding `09_seletopclusts` directory.

While these simple text files can be easily checked from the command line already, they might be cumbersome to read.
For that reason, we have developed a post-processing analysis that automatically generates html reports for all `caprieval` steps in the workflow.
These are located in the respective `analysis/XX_caprieval` directories and can be viewed using your favorite web browser.




---

## Cluster statistics

Let us now analyse the docking results. Use for that either your own run or a pre-calculated run provided in the `runs` directory.
The `analysis/10_caprieval_analysis` directory of the respective run directory contains various html files. To visualise the report directly in your browser, download the files to your computer and open those in a web browser.  The `report.html` contains interactive plots that may take some time to generate.

On the top of the page, you will see a table that summarises the cluster statistics (taken from the `capri_clt.tsv` file).
The columns (corresponding to the various clusters) are sorted by default on the cluster rank, which is based on the HADDOCK score.


In [ ]:
#@title View the cluster statistics table
import json
import re
from plotly.offline import iplot
from IPython.display import display, HTML
import pandas as pd

# Specify the path to your HTML file
html_file_path = PROJECT_DIR / "runs" / "run1" / "analysis" / "10_caprieval_analysis/report.html"

# Read the HTML file
with open(html_file_path, 'r') as f:
    html_content = f.read()

# Use a regular expression to find the script tag with id="datatable2" and extract its content
match_table = re.search(r'<script id="datatable2" type="application/json">\s*(.*?)\s*</script>', html_content, re.DOTALL)

if match_table:
    json_data_str_table = match_table.group(1)
    # Load the JSON data
    table_data = json.loads(json_data_str_table)

    # Extract the 'clusters' list from the JSON data
    clusters_data = table_data.get('clusters', [])

    # Create a pandas DataFrame from the clusters data
    df = pd.json_normalize(clusters_data)

    # Drop the columns related to best cluster files
    cols_to_drop = ['under_eval', 'total', 'rmsd', 'best1', 'best2', 'best3', 'best4', 'ilrmsd.mean', 'ilrmsd.std', 'air.mean', 'air.std', 'desolv.mean', 'desolv.std', 'elec.mean', 'elec.std', 'vdw.mean', 'vdw.std'] # Add more if there are more 'best' columns
    df = df.drop(columns=[col for col in cols_to_drop if col in df.columns])

    # Display the DataFrame
    display(df)
else:
    print("Could not find the script tag with id='datatable2' in the HTML file.")


Inspect the final cluster statistics.

__*How many clusters have been generated?*__

__*Look at the score of the first few clusters: Are they significantly different if you consider their average scores and standard deviations?*__

Since for this tutorial we have at hand the crystal structure of the complex, we provided it as reference to the `caprieval` modules.
This means that the iRMSD, lRMSD, Fnat and DockQ statistics report on the quality of the docked model compared to the reference crystal structure.

CAPRI is using the following criteria to assess the quality of a model:

* **Unacceptable**:    i-RMSD >4Å or Fnat < 0.1
* **Acceptable**:      4Å <=i-RMSD < 2Å and Fnat > 0.1
* **Medium**:          2Å <=i-RMSD < 1Å and Fnat > 0.3
* **High**:            i-RMSD < 1Å and Fnat > 0.5

__*How many clusters of acceptable or better quality have been generate according to CAPRI criteria?*__

__*What is the rank of the best cluster generated?*__

__*What is the cluster ID of the best cluster generated?*__

The cluster ID is related to the size of the cluster with cluster 1 being the largest one.

__*What is the rank of the first acceptable of better cluster generated?*__

## Visualizing the scores and their components


Next to the cluster statistic table shown above, the `report.html` file also contains a variety of plots displaying the HADDOCK score
and its components against various CAPRI metrics (i-RMSD, l-RMSD,  Fnat, Dock-Q) with a color-coded representation of the clusters.
These are interactive plots. A menu on the top right of the first row (you might have to scroll to the rigth to see it)
allows you to zoom in and out in the plots and turn on and off clusters.

To view theses plots in the notebook execute the following cell.

__*Examine the plots (remember here that higher DockQ values and lower i-RMSD values correspond to better models)*__


In [ ]:
#@title View the various statistics plots
import json
import re
from plotly.offline import iplot
from IPython.display import display, HTML
import pandas as pd

# Specify the path to your HTML file
html_file_path = PROJECT_DIR  / "runs" / "run1" / "analysis" / "10_caprieval_analysis/report.html"

# Read the HTML file
with open(html_file_path, 'r') as f:
    html_content = f.read()

# Use a regular expression to find the script tag with id="data1" and extract its content
match_plot = re.search(r'<script id="data1" type="application/json">\s*(.*?)\s*</script>', html_content, re.DOTALL)

if match_plot:
    json_data_str_plot = match_plot.group(1)
    # Load the JSON data
    plot_data = json.loads(json_data_str_plot)

    # Set width and height to None to make the plot responsive
    if 'layout' in plot_data:
        plot_data['layout']['width'] = None
        #plot_data['layout']['height'] = None

    # Display the plot using iplot
    iplot(plot_data)
else:
    print("Could not find the script tag with id='data1' in the HTML file.")



Finally, the report also shows plots of the cluster statistics (distributions of values per cluster ordered according to their HADDOCK rank).

To view those distribution plots execute the next cell.

__*For this antibody-antigen case, which of the score components correlates best with the quality of the models?*__



In [ ]:
#@title View the distribution plots
import json
import re
from plotly.offline import iplot
from IPython.display import display, HTML
import pandas as pd

# Specify the path to your HTML file
html_file_path = PROJECT_DIR / "runs" / "run1" / "analysis" / "10_caprieval_analysis/report.html"

# Read the HTML file
with open(html_file_path, 'r') as f:
    html_content = f.read()

# Use a regular expression to find the script tag with id="data1" and extract its content
match_plot = re.search(r'<script id="data2" type="application/json">\s*(.*?)\s*</script>', html_content, re.DOTALL)

if match_plot:
    json_data_str_plot = match_plot.group(1)
    # Load the JSON data
    plot_data = json.loads(json_data_str_plot)
    # Set width and height to None to make the plot responsive
    if 'layout' in plot_data:
        plot_data['layout']['width'] = None
        #plot_data['layout']['height'] = None

    # Display the plot using iplot
    iplot(plot_data)
else:
    print("Could not find the script tag with id='data1' in the HTML file.")



---



## Some single structure analysis


Going back to command line analysis, we are providing in the `scripts` directory a simple script that extracts some model statistics for acceptable or better models from the `caprieval` steps.
To use it, simply call the script with as argument the run directory you want to analyze, e.g.:

```
scripts/extract-capri-stats.sh runs/run1
```

_**Note**_ that this kind of analysis only makes sense when we know the reference complex and for benchmarking / performance analysis purposes.



In [ ]:
!scripts/extract-capri-stats.sh runs/run1


Look at the single structure statistics provided by the script.

__*How does the quality of the best model changes after flexible refinement? Consider here the various metrics.*__

<details style="background-color:#DAE4E7">
  <summary style="bold">
    <i>Answer</i> <i class="material-icons">expand_more</i>
  </summary>
  <p>
  In terms of iRMSD values, we only observe very small differences in the best model.
  The fraction of native contacts and the DockQ scores are however improving much more after flexible refinement but increases again slightly after final minimisation.
  All this will of course depend on how different are the bound and unbound conformations and the amount of data used to drive the docking process.
  In general, from our experience, the more and better data at hand, the larger the conformational changes that can be induced.
  </p>
</details>
<br>

__*Is the best model always ranked first?*__

<details style="background-color:#DAE4E7">
  <summary style="bold">
    <i>Answer</i> <i class="material-icons">expand_more</i>
  </summary>
  <p>
    This is not the case. The scoring function is not perfect, but does a reasonable job at ranking models of acceptable or better quality on top in this case.
  </p>
</details>
<br>

**_Note_**: A similar script to extract cluster statistics is available in the `scripts` directory as `extract-capri-stats-clt.sh`.





---



## Contacts analysis

We have added a contact analysis module to HADDOCK3 that generates for each cluster both a contact matrix of the entire system showing all contacts within a 4.5Å cutoff and a chord chart representation of intermolecular contacts.

In the current workflow we run, those files can be found in the `11_contactmap` directory. These are again html files with interactive plots (hover with your mouse over the plots).

This file taken from the pre-computed run can be visualized by executing the next cell which will visualize the contacts of the first ranked cluster (cluster 2).

__*Can you identify which residue(s) make(s) the most intermolecular contacts?*__



In [ ]:
#@title View the contact chord chart
import json
import re
from plotly.offline import iplot
from IPython.display import display, HTML

# Specify the path to the HTML file containing the chord chart
html_file_path = PROJECT_DIR / "runs" / "run1" / "11_contactmap" / "cluster2_rank1_chordchart.html"

# Read the HTML file
with open(html_file_path, 'r') as f:
    html_content = f.read()

# Use a regular expression to find the script tag containing the plot data
# The id of the script tag might vary, so a more general pattern is used
# We assume the plot data is within a script tag with type="application/json"
match_plot = re.search(r'<script.*?type="application/json".*?>\s*(.*?)\s*</script>', html_content, re.DOTALL)

if match_plot:
    json_data_str_plot = match_plot.group(1)
    # Load the JSON data
    plot_data = json.loads(json_data_str_plot)
    # Display the plot using iplot
    iplot(plot_data)
else:
    print(f"Could not find the plot data in the HTML file at {html_file_path}")



---



## Visualization of the models

We can visualize the models from the various clusters by execting the next cell (edit it to change the model you want to visualize). The model visualised here is the top ranked model of the top ranker cluster (cluster 2).

For more detailed visualisation using PyMOL (after downloading the results to your local computer refer to our [online tutorial](https://www.bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/#visualization-of-the-models).




Let’s now check if the active residues which we have defined (the paratope and epitope) are actually part of the interface. For this execute the next cell.

The active residues of the antibody (paratope) are shown in red, those of the antigen (epitope) in orange and the passive residues of the antigen in green.

__*Are the residues of the paratope and NMR epitope at the interface?*__



In [ ]:
#@title Visualize the active and passive residues in the model
import py3Dmol
import gzip
import os

# Create a 3Dmol viewer object
view = py3Dmol.view(width=800, height=600)

# Specify the path to your PDB file (can be .pdb or .pdb.gz)
pdb_file_path = PROJECT_DIR / "runs" / "run1" / "09_seletopclusts" / "cluster_2_model_1.pdb.gz"

# Check if the file exists
if not os.path.exists(pdb_file_path):
    print(f"Error: File not found at {pdb_file_path}")
else:
    # Check if the file is gzipped
    if str(pdb_file_path).endswith('.gz'):
        with gzip.open(pdb_file_path, 'rt') as f:
            view.addModel(f.read(), 'pdb')
    else:
        with open(pdb_file_path, 'r') as f:
            view.addModel(f.read(), 'pdb')

    view.setStyle({'cartoon': {'color': 'white'}})
    view.addStyle({'stick': {'color': 'white'}})
    view.addStyle({'resi': [31,32,33,34,35,52,54,55,56,100,101,102,103,104,105,106,151,152,169,170,173,211,212,213,214,216], 'chain': 'A'},{'cartoon':{'color':'red'}})
    view.addStyle({'resi': [31,32,33,34,35,52,54,55,56,100,101,102,103,104,105,106,151,152,169,170,173,211,212,213,214,216], 'chain': 'A'},{'stick':{'color':'red'}})
    view.addStyle({'resi': [72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117], 'chain': 'B'},{'cartoon':{'color':'organge'}})
    view.addStyle({'resi': [72,73,74,75,81,83,84,89,90,92,94,96,97,98,115,116,117], 'chain': 'B'},{'cartoon':{'color':'orange'}})
    view.addStyle({'resi': [3,24,46,47,48,50,66,76,77,79,80,82,86,87,88,91,93,95,118,119,120], 'chain': 'B'},{'cartoon':{'color':'green'}})
    view.addStyle({'resi': [3,24,46,47,48,50,66,76,77,79,80,82,86,87,88,91,93,95,118,119,120], 'chain': 'B'},{'stick':{'color':'green'}})
    view.addSurface(py3Dmol.SAS, {'opacity':0.4, 'color':'white'})
    view.zoomTo()
    view.show()

In [ ]:
#@title View the top ranking model from the best cluster
#@markdown It will be shown superimposed onto the reference structure
# Define paths to PDB files to be aligned
# Here we are overlaying the reference and 1st model from the 1st cluster
path1 = PROJECT_DIR / "pdbs" / "4G6M_matched.pdb"
path2 = PROJECT_DIR / "runs" / "run1" / "09_seletopclusts" / "cluster_1_model_1.pdb.gz"

# Align molecules by chain and display the result.
# For the full alignement put all chain IDs in the `chains` variable.
align_full(str(path1), str(path2), chains = ['A','B'])

In our [online tutorial](https://www.bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/#visualization-of-the-models) you can find instructions to compare the models with the reference structure using PyMOL.




---


# Conclusions

We have demonstrated the usage of HADDOCK3 in an antibody-antigen docking scenario making use of the paratope information on the antibody side (i.e. no prior experimental information, but computational predictions) and an NMR-mapped epitope for the antigen.
Compared to the static HADDOCK2.X workflow, the modularity and flexibility of HADDOCK3 allow to customise the docking protocols and to run a deeper analysis of the results.
HADDOCK3's intrinsic flexibility can be used to improve the performance of antibody-antigen modelling compared to the results we presented in our
[Structure 2020](https://doi.org/10.1016/j.str.2019.10.011) article and in the [related HADDOCK2.4 tutorial](/education/HADDOCK24/HADDOCK24-antibody-antigen).

<br>
<br>







---




# Congratulations! 🎉

You have completed this tutorial. If you have any questions or suggestions, feel free to contact us via email or asking a question through our [support center](https://ask.bioexcel.eu).

And check also our [education web page](https://bonvinlab.org/education) where you will find more tutorials!

But there is more: Check the BONUSES!




---



---

# BONUS1: Dissecting the interface energetics: what is the impact of a single mutation?

Mutations at the binding interfaces can have widely varying effects on binding affinity - some may be negligible, while others can significantly strengthen or weaken the interaction. Exploring these mutations helps identify critical amino acids for redesigning structurally characterized protein-protein interfaces, which paves the way for developing protein-based therapeutics to deal with a diverse range of diseases.
To pinpoint such amino acids positions, the residues across the protein interaction surfaces are either randomly or strategically mutated. Scanning mutations in this manner is experimentally costly. Therefore, computational methods have been developed to estimate the impact of an interfacial mutation on protein-protein interactions.
These computational methods come in two main flavours. One involves rigorous free energy calculations, and, while highly accurate, these methods are computationally expensive. The other category includes faster, approximate approaches that predict changes in binding energy using statistical potentials, machine learning, empirical scoring functions etc. Though less precise, these faster methods are practical for large-scale screening and early-stage analysis. In this bonus exercise, we will take a look at two quick ways of estimating the effect of a single mutation in the interface.






---

## PROT-ON and haddock3-scoring to inspect a single mutation

PROT-ON (Structure-based detection of designer mutations in PROTein-protein interface mutatiONs) is a tool and [online server](http://proton.tools.ibg.edu.tr:8001/about) that scans all possible interfacial mutations and **predicts ΔΔG score** by using EvoEF1 (active in both on the web server and stand-alone versions) or FoldX (active only in the stand-alone version) with the aim of finding the most mutable positions. The original publication describing PROT-ON can be found [here](https://www.frontiersin.org/journals/molecular-biosciences/articles/10.3389/fmolb.2023.1063971/full).

Here we will use PROT-ON to analyse the interface of our antibody-antigen complex. For that, we will use the provided matched reference structure (`4G6M-matched.pdb`) in which both chains of the antibody have the same chainID (A), which allows us to analyse all interface residues of the antibody at once.

__Note:__ Pre-calculated PROT-ON results for this system can be accessed [here](http://proton.tools.ibg.edu.tr:8001/result/ebcdec31308c46acb82e8010f7f21df1).


Connect to the PROT-ON server page (link above) and fill in the following fields:


```
Specify your run name --> 4G6M_matched
```

```
Choose / Upload your protein complex --> Select the provided _4G6M-matched.pdb_ file
```

```
Which dimer chains should be analyzed --> Select chain A for the 1st molecule and B for the 2nd
```

```
Pick the monomer for mutational scanning --> Select the first molecule - the antibody (toggle the switch ON under the chain A)
```

```
Click on the Submit button
```

Your run should complete in 5-10 minutes. Once finished, you will be presented with a result page summarising the most depleting (ones that decrease the binding affinity) and most enriching (ones that increase the binding affinity) mutations.

__*Which possible mutation would you propose to improve the binding affinity of the antibody?*__

<details style="background-color:#DAE4E7">
  <summary style="bold">
    <b><i>See answer</i></b> <i class="material-icons">expand_more</i>
  </summary>
The most enriching mutation is S150W with a -3.69 ΔΔG score.
</details>
<br>

Visualise the propose mutated residue in the 3D model (for this run the next cell). The mutated residue (change the residue selection if needed) will be displayed as spheres.

__*Can you rationalise why it might increase the affinity?*__


In [ ]:
#@title Visualise the most enriching (increasing the interaction energy) mutation
import py3Dmol
import gzip
import os

# Create a 3Dmol viewer object
view = py3Dmol.view(width=800, height=600)

# Specify the path to your PDB file (can be .pdb or .pdb.gz)
pdb_file_path = PROJECT_DIR / "pdbs" / "4G6M_matched.pdb"

# Check if the file exists
if not os.path.exists(pdb_file_path):
    print(f"Error: File not found at {pdb_file_path}")
else:
    # Check if the file is gzipped
    if str(pdb_file_path).endswith('.gz'):
        with gzip.open(pdb_file_path, 'rt') as f:
            view.addModel(f.read(), 'pdb')
    else:
        with open(pdb_file_path, 'r') as f:
            view.addModel(f.read(), 'pdb')

    view.setStyle({'cartoon': {'color': 'white'}})
    view.addStyle({'stick': {}})
    view.addStyle({'resi': '150', 'chain': 'A'},{'sphere':{}})
    view.addSurface(py3Dmol.SAS, {'opacity':0.4, 'color':'white'})
    view.zoomTo()
    view.show()

With HADDOCK3, it is possible to take a step further. To perform the mutation, simply rename the desired residue and score such model - HADDOCK will take care of the topology regardless of the side chain differences and energy minimisation of the model. To do so, change the name of the residue to be mutated in the _4G6M-matched.pdb_ PDB file and save it to a new file. This can be done in the terminal with the following command:

```
sed 's/SER\ A\ 150/TRP\ A\ 150/g' pdbs/4G6M_matched.pdb > 4G6M_matched_S150W.pdb
```

Next, score the mutant using the command-line tool `haddock3-score`.
This tool performs a short workflow composed of the `topoaa` and `emscoring` modules. Use flag `--outputpdb` to save energy-minimized model:

```
haddock3-score 4G6M_matched_S150W.pdb --outputpdb
```

Use _haddock3-score_ to calculate the score of the wild type, 4G6M-matched.pdb, and the mutation, 4G6M_matched_S150W.pdb.



In [ ]:
!sed 's/SER\ A\ 150/TRP\ A\ 150/g' pdbs/4G6M_matched.pdb > 4G6M_matched_S150W.pdb
!haddock3-score pdbs/4G6M_matched.pdb --outputpdb
!haddock3-score 4G6M_matched_S150W.pdb --outputpdb


__*Do you see a difference between wild-type and mutant scores?
Might such single-residue mutation affect the binding affinity?*__


Inspect the energy-minimized mutant model (4G6M_matched_S150W_hs.pdb) visually (for this run the next cell).

__*Can you rationalise why such a mutation might increase the affinity?*__


<details style="background-color:#DAE4E7">
  <summary style="bold">
    <b><i>Zoom in on the mutated residue</i></b>
  </summary>
  <figure style="text-align: center;">
   <img width="100%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/mutant-stacking.png">
   <center>
   <i>TRP150 is stacking with TYR24 </i>
   </center>
  </figure>
  <br>
</details>
<br>



In [ ]:
#@title Visualise the S150W mutation
import py3Dmol
import gzip
import os

# Create a 3Dmol viewer object
view = py3Dmol.view(width=800, height=600)

# Specify the path to your PDB file (can be .pdb or .pdb.gz)
pdb_file_path = PROJECT_DIR / "pdbs" / "4G6M_matched_S150W_hs.pdb"

# Check if the file exists
if not os.path.exists(pdb_file_path):
    print(f"Error: File not found at {pdb_file_path}")
else:
    # Check if the file is gzipped
    if str(pdb_file_path).endswith('.gz'):
        with gzip.open(pdb_file_path, 'rt') as f:
            view.addModel(f.read(), 'pdb')
    else:
        with open(pdb_file_path, 'r') as f:
            view.addModel(f.read(), 'pdb')

    view.setStyle({'cartoon': {'color': 'white'}})
    view.addStyle({'stick': {}})
    view.addStyle({'resi': '150', 'chain': 'A'},{'sphere':{}})
    view.addSurface(py3Dmol.SAS, {'opacity':0.4, 'color':'white'})
    view.zoomTo()
    view.show()




---


## Alanine Scanning module

Another way of exploring interface energetics is by using the `alascan` module of HADDOCK3. `alascan` stands for "Alanine Scanning module".

This module is capable of mutating interface residues to Alanine and calculating the **Δ HADDOCK score** between the wild-type and mutant, thus providing a measure of the impact of each individual mutation. It is possible to scan all interface residues one by one or limit this scanning to a selected by user set of residues. By default, the mutation to Alanine is performed, as its side chain is just a methyl group, so side chain perturbations are minimal, as well as possible secondary structure changes. It is possible to perform the mutation to any other amino acid type - at your own risk, as such mutations may introduce structural uncertainty.

**Important**: 1/ `alascan` calculates the difference between wild-type score vs mutant score, i.e. positive `Δscore` indicative of the enriched (stronger) binding and negative `Δscore` is indicative of the depleted (weaker) binding; 2/ Inside `alascan`, a short energy minimization of an input structure is performed, i.e. there's no need to include an additional refinement module prior to `alascan`.

Here is an example of the workflow to scan interface energetics:

```
# ====================================================================
# Scanning interface residues with haddock3
# ====================================================================

# directory in which the scoring will be done
run_dir = "run-energetics-alascan"

# compute mode
mode = "local"
ncores = 50

# Post-processing to generate statistics and plots
postprocess = true
clean = true

molecules =  [
    "pdbs/4G6M_matched.pdb",
    ]

# ====================================================================
# Parameters for each stage are defined below
# ====================================================================
[topoaa]

[emref]

[alascan]
# mutate each interface residue to Alanine
scan_residue = 'ALA'
# generate plot of delta score and its components per each mutation
plot = true

# ====================================================================
```


An alanine scaning scenario configuration file is provided in the `workflows/` directory as `interaction-energetics.cfg`, and precomputed results are in `runs/run-energetics-alascan`.


In [ ]:
#@title Run the haddock3 alanine scanning workflow
!haddock3 workflows/interaction-energetics.cfg

The output folder contains, among others, a directory titled `2_alascan` with a file `scan_clt_unclustered.tsv` that lists each mutation, corresponding score and individual terms:

```
################################################################################
# `alascan` cluster results for cluster unclustered
# reported values are the average for the cluster
#
# z_score is calculated with respect to the mean values of all residues
################################################################################
chain   resid   resname full_resname    delta_score     delta_score_std delta_vdw       delta_vdw_std   delta_elec      delta_elec_std  delta_desolv    delta_desolv_std        delta_bsa       delta_bsa_std   frac_pres       z_score
A       32      SER     A-32-SER        4.71    0.00    1.49    0.00    5.66    0.00    2.08    0.00    24.02   0.00    1.00    1.18
A       33      GLY     A-33-GLY        2.55    0.00    3.23    0.00    0.03    0.00    -0.69   0.00    16.31   0.00    1.00    0.85
A       54      TRP     A-54-TRP        0.16    0.00    -2.11   0.00    9.29    0.00    0.41    0.00    12.26   0.00    1.00    0.49
A       55      TRP     A-55-TRP        -5.45   0.00    -2.92   0.00    5.52    0.00    -3.64   0.00    59.08   0.00    1.00    -0.35
A       56      ASP     A-56-ASP        -12.38  0.00    1.26    0.00    -81.09  0.00    2.58    0.00    12.53   0.00    1.00    -1.39
A       58      ASP     A-58-ASP        -21.75  0.00    2.24    0.00    -135.74 0.00    3.16    0.00    39.68   0.00    1.00    -2.80
A       59      GLU     A-59-GLU        0.19    0.00    0.11    0.00    -1.14   0.00    0.31    0.00    12.42   0.00    1.00    0.50
A       60      SER     A-60-SER        2.58    0.00    3.12    0.00    -10.76  0.00    1.62    0.00    17.58   0.00    1.00    0.86
...
```

__*Can you identify the most enriching/depleting mutation of each chain? *__

You can use an additional script `scripts/get-alascan-extrema.sh` to check your answer:


In [ ]:
!bash scripts/get-alascan-extrema.sh runs/run-energetics-alascan/2_alascan/scan_clt_unclustered.tsv


Mutation of the residue ASP58 turned out to be the most depleting within chain A.

Let us visualise the impact of each mutated residue. This can be done by looking at the interactive plot generated in the `2_alascan` directory called `scan_clt_unclustered.html`.

In the generated interactive plot, negative values are indicative of mutations that should destabilize the complex.


In [ ]:
#@title Visualise the results of the alanine scanning
import json
import re
from plotly.offline import iplot
from IPython.display import display, HTML

# Specify the path to the HTML file
html_file_path = PROJECT_DIR / "runs" / "run-energetics-alascan" / "2_alascan" / "scan_clt_unclustered.html"

# Read the HTML file
try:
    with open(html_file_path, 'r') as f:
        html_content = f.read()

    # Use a regular expression to find a script tag with type="application/json" and extract its content
    # The id of the script tag might vary, so a general pattern is used
    match_plot = re.search(r'<script id="data1" type="application/json">\s*(.*?)\s*</script>', html_content, re.DOTALL)

    if match_plot:
        json_data_str_plot = match_plot.group(1)
        # Load the JSON data
        plot_data = json.loads(json_data_str_plot)

        # Set width and height to None to make the plot responsive
        if 'layout' in plot_data:
            plot_data['layout']['width'] = None
            plot_data['layout']['height'] = None

        # Display the plot using iplot
        iplot(plot_data)
    else:
        print("Could not find JSON plot data in the HTML file.")

except FileNotFoundError:
    print(f"Error: File not found at {html_file_path}")
except Exception as e:
    print(f"An error occurred: {e}")

__*Can you identify the most enriching/depleting mutation of each chain?*__

You can use an additional script /scripts/get-alascan-extrema.sh to check your answer:


In [ ]:
!bash scripts/get-alascan-extrema.sh runs/run-energetics-alascan/2_alascan/scan_clt_unclustered.tsv

Mutation of the residue ASP58 turned out to be the most depleting within chain A.


In [ ]:
#@title Visualise the Asp58 environemnt
import py3Dmol
import gzip
import os

# Create a 3Dmol viewer object
view = py3Dmol.view(width=800, height=600)

# Specify the path to your PDB file (can be .pdb or .pdb.gz)
pdb_file_path = PROJECT_DIR / "pdbs" / "4G6M_matched_S150W_hs.pdb"

# Check if the file exists
if not os.path.exists(pdb_file_path):
    print(f"Error: File not found at {pdb_file_path}")
else:
    # Check if the file is gzipped
    if str(pdb_file_path).endswith('.gz'):
        with gzip.open(pdb_file_path, 'rt') as f:
            view.addModel(f.read(), 'pdb')
    else:
        with open(pdb_file_path, 'r') as f:
            view.addModel(f.read(), 'pdb')

    view.setStyle({'cartoon': {'color': 'white'}})
    view.addStyle({'stick': {}})
    view.addStyle({'resi': '58', 'chain': 'A'},{'sphere':{}})
    view.addSurface(py3Dmol.SAS, {'opacity':0.4, 'color':'white'})
    view.zoomTo()
    view.show()

You should be able to see that ASP58 is close to two lysines: LYS98, and LYS94.

<figure style="text-align: center;">
  <img width="50%" src="https://bonvinlab.org/education/HADDOCK3/HADDOCK3-antibody-antigen/asp58_contacts.png">
  <center>
  <i>ASP58 makes h-bonds with two neighbouring residues</i>
  </center>
</figure>





---

---

# BONUS 2: Running a scoring scenario

This section demonstrates the use of HADDOCK3 to score the various models obtained using HADDOCK and AlphaFold predictions and observe if the HADDOCK scoring function is able to detect the quality of the models.

To this end the following workflow is defined:

1. Generate the topologies for the various models.
2. Energy Minimise all complexes.
3. Cluster the models using Fraction of Common Contacts:
  - set the parameter `min_population` to 1 so that all models, including singletons (models that do not cluster with any others), will be forwarded to the next steps.
  - set the parameter `plot_matrix` to true to generate a matrix of the clusters for a visual representation.
4. Comparison of the models with the reference complex `4G6M_matched.pdb` using CAPRI criteria.

For this, two ensembles must be scored and one structure will be used as a reference. You can find them in the `pdbs/` directory:
- `07_emref_and_top5af2_ensemble.pdb`: An ensemble of models obtained from the ensemble run, combined with top5 AlphaFold2 predictions.
- `af3server_15052024_top5ens.pdb`: An ensemble of top5 AlphaFold3 predictions.
- `4G6M_matched.pdb`: The reference structure for quality assessments.


```
# ====================================================================
# Antibody-antigen docking example with restraints from the antibody
# paratope to the NMR-identified epitope on the antigen
# ====================================================================
run_dir = "scoring-haddock3-alphafold2and3-ensemble"

molecules =  [
    "pdbs/haddock3-ens-emref-ensemble.pdb",
    "pdbs/af2-models.pdb",
    "pdbs/af3-models.pdb",
    ]

# ====================================================================
# Parameters for each stage are defined below
# ====================================================================

[topoaa]

[emscoring]

[clustfcc]
# Reduce the min_population to define a cluster to 1 so that models
# that do not cluster with any other will define singlotons
min_population = 1
# Generate a matrix of the clusters
plot_matrix = true

[caprieval]
reference_fname = "4G6M_matched.pdb"

# ====================================================================
```


A scoring scenario configuration file is provided in the `workflows/` directory as `scoring-antibody-antigen.cfg, precomputed results in `runs/run-scoring`.


In [ ]:
#@title Run the haddock3 scoring workflow
!haddock3 workflows/scoring-antibody-antigen.cfg


You can look at the `capri_ss.tsv` file in the `4_caprieval` directory. It contains the energy minimised statistics:

```
model   md5     caprieval_rank  score   irmsd   fnat    lrmsd   ilrmsd  dockq   rmsd    cluster_id      cluster_ranking model-cluster_ranking   air     angles  bonds   bsa     cdih    coup    dani    desolv  dihe    elec    improper        rdcs    rg      sym     total   vdw     vean    xpcs
../1_emscoring/emscoring_82.pdb -       1       -157.149        0.910   0.897   2.201   1.456   0.855   1.016   2       1       1       0.000   296.229 64.664  2000.130        0.000   0.000   0.000   7.345   2068.230        -599.183        84.513  0.000   0.000   0.000   -643.841        -44.658      0.000   0.000
../1_emscoring/emscoring_2.pdb  -       2       -156.452        0.880   0.948   1.949   1.355   0.881   0.989   2       1       2       0.000   298.570 59.587  1914.860        0.000   0.000   0.000   3.125   2052.880        -504.372        85.779  0.000   0.000   0.000   -563.075        -58.703      0.000   0.000
../1_emscoring/emscoring_64.pdb -       3       -138.214        1.052   0.914   3.039   1.955   0.824   1.294   2       1       3       0.000   286.361 53.633  1784.350        0.000   0.000   0.000   -2.359  2040.230        -424.542        77.017  0.000   0.000   0.000   -475.489        -50.947      0.000   0.000
../1_emscoring/emscoring_40.pdb -       4       -135.230        1.085   0.897   1.866   1.756   0.836   1.144   2       1       4       0.000   299.170 56.868  1875.210        0.000   0.000   0.000   3.490   2034.370        -429.067        87.524  0.000   0.000   0.000   -481.973        -52.906      0.000   0.000
../1_emscoring/emscoring_37.pdb -       5       -134.569        13.624  0.069   22.589  21.764  0.068   13.881  5       2       1       0.000   305.568 67.872  1802.890        0.000   0.000   0.000   6.081   2050.950        -426.815        93.743  0.000   0.000   0.000   -482.102        -55.287      0.000   0.000
../1_emscoring/emscoring_22.pdb -       6       -129.571        4.855   0.155   11.612  10.609  0.197   4.234   8       3       1       0.000   309.609 62.278  1591.470        0.000   0.000   0.000   8.217   2058.920        -345.081        86.168  0.000   0.000   0.000   -413.853        -68.772      0.000   0.000
...
```

__*
Did the HADDOCK scoring do a good job at putting the best models on top (consider for example the DockQ score)?*__

The `emscoring` module renames all models, which makes it difficult to know what was the original model.
You can however trace back a model to its original file by looking into the `traceback/traceback.tsv` file:

```
00_topo1        1_emscoring     1_emscoring_rank
emref_9_from_haddock3-ens-emref-ensemble_83_haddock.psf   emscoring_82.pdb        1
emref_10_from_haddock3-ens-emref-ensemble_2_haddock.psf   emscoring_2.pdb        2
emref_7_from_haddock3-ens-emref-ensemble_67_haddock.psf   emscoring_64.pdb        3
emref_5_from_haddock3-ens-emref-ensemble_45_haddock.psf   emscoring_40.pdb        4
emref_47_from_haddock3-ens-emref-ensemble_42_haddock.psf  emscoring_37.pdb        5
emref_35_from_haddock3-ens-emref-ensemble_29_haddock.psf  emscoring_22.pdb        6
...
```

__*Try to locate the AlphaFold2 and AlphaFold3 models (their filenames start with _abag_test_ and _af3server_, respectively)*__

A simple way to extra this information is to use `grep`:


In [ ]:
!grep abag runs/run-scoring/traceback/traceback.tsv


<details style="background-color:#DAE4E7">
  <summary style="font-weight: bold">
    <i>See the output of grep for the AlphaFold2 models</i>
  <br>
  </summary>
  <pre>
> grep abag traceback.tsv
abagtest_2d03e_unrelaxed_rank_001_alphafold2_multimer_v3_model_3_seed_000_from_af2-models_1_haddock.psf	emscoring_84.pdb	86
abagtest_2d03e_unrelaxed_rank_005_alphafold2_multimer_v3_model_2_seed_000_from_af2-models_5_haddock.psf	emscoring_88.pdb	90
abagtest_2d03e_unrelaxed_rank_004_alphafold2_multimer_v3_model_4_seed_000_from_af2-models_4_haddock.psf	emscoring_87.pdb	91
abagtest_2d03e_unrelaxed_rank_003_alphafold2_multimer_v3_model_1_seed_000_from_af2-models_3_haddock.psf	emscoring_86.pdb	92
abagtest_2d03e_unrelaxed_rank_002_alphafold2_multimer_v3_model_5_seed_000_from_af2-models_2_haddock.psf	emscoring_85.pdb	93
  </pre>
</details>
<br>


In [ ]:
!grep af3server runs/run-scoring/traceback/traceback.tsv


<details style="background-color:#DAE4E7">
  <summary style="font-weight: bold">
    <i>See the output of grep for the AlphaFold3 models</i>
  <br>
  </summary>
  <pre>
> grep abag traceback.tsv
af3server_15052024_2_ready_from_af3-models_2_haddock.psf	emscoring_90.pdb	40
af3server_15052024_1_ready_from_af3-models_1_haddock.psf	emscoring_89.pdb	81
af3server_15052024_4_ready_from_af3-models_4_haddock.psf	emscoring_92.pdb	87
af3server_15052024_3_ready_from_af3-models_3_haddock.psf	emscoring_91.pdb	88
af3server_15052024_5_ready_from_af3-models_5_haddock.psf	emscoring_93.pdb	89
  </pre>
</details>
<br>

__*What are their ranks?*__

We have already seen in the previous section that none of the AlphaFold models were close to the real complex. This is however also the case for some of the HADDOCK models. Still the AlpaFold models score very badly, toward the end of the ranked list of models.

__*Having found their ranks, can you figure out from the statistics in _capri_ss.tsv_ which component of the HADDOCK score causes in particular this bad scoring?*__


<details style="background-color:#DAE4E7">
  <summary style="font-weight: bold">
    <i>See the answer</i>
  <br>
  </summary>
  <p> The bottom eight models (the worst ranking ones) are all AlphaFold3/2 models. Looking at the componenents of the score
  (some were left out in the table below for simplicity) one can see that it is mainly the van der Waals energy that causes the high scores,
  which is indicative of clashes in the models.</p>
  <pre>
model               md5 caprieval_rank  score    irmsd   fnat    lrmsd   ilrmsd  dockq   rmsd    bsa        desolv    elec      vdw vean    xpcs
...
../1_emscoring/emscoring_84.pdb -   86  -67.914  11.123  0.000   22.413  18.626  0.048   12.213  3535.520   -67.537  -150.913    29.806
../1_emscoring/emscoring_92.pdb -   87  -63.263  11.426  0.000   22.104  21.035  0.049   11.048  1383.920    -9.924   -88.656   -35.607
../1_emscoring/emscoring_91.pdb -   88  -50.990  13.665  0.000   23.793  22.150  0.042   13.796  1492.150    -8.962  -167.236    -8.581
../1_emscoring/emscoring_93.pdb -   89  -46.871   6.644  0.000   10.617  11.333  0.146   6.455   1740.990    -8.906   -35.623   -30.841
../1_emscoring/emscoring_88.pdb -   90   48.283  12.919  0.000   20.484  19.885  0.053   14.706  3914.250   -68.786  -129.461   142.961
../1_emscoring/emscoring_87.pdb -   91  180.468  12.447  0.000   22.153  19.299  0.048   14.160  3639.430   -66.857  -240.130   295.351
../1_emscoring/emscoring_86.pdb -   92  240.307  12.572  0.000   21.662  19.799  0.049   14.187  3535.820   -69.380  -154.703   340.628
../1_emscoring/emscoring_85.pdb -   93  781.210  15.174  0.000   23.497  24.993  0.042   17.151  3278.340   -61.261   -86.026   859.677
  </pre>
</details>
<br>




---



---




# Congratulations! 🎉

You have completed this tutorial. If you have any questions or suggestions, feel free to contact us via email or asking a question through our [support center](https://ask.bioexcel.eu).

And check also our [education web page](https://bonvinlab.org/education) where you will find more tutorials!




---



---

#Save your results to Google Drive

In order to save the entire tutorial directory with your results run the following cell.

In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the source directory
source_dir = './HADDOCK3-antibody-antigen-notebook'

# Define the destination directory in Google Drive
destination_dir = '/content/drive/MyDrive/'

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Move the directory
!cd ../; mv "$source_dir" "$destination_dir"

print(f"Moved '{source_dir}' to '{destination_dir}'")